<a href="https://colab.research.google.com/github/niikun/AI_Agent_Dev/blob/main/Day1_%EF%BC%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install openai pypdf gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.0/329.0 kB 9.9 MB/s eta 0:00:00


In [5]:
import os
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [6]:
openai = OpenAI()

In [18]:
PATH = "/content/drive/MyDrive/Free_learning/AI_agent2B/me"
linkedin = PATH + "/linkedin.pdf"

reader = PdfReader(linkedin)
linkedin = ""

for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

print(len(linkedin))

8256


In [19]:
linkedin

"\xa0 \xa0\nContact\ned.donner@gmail.com\nwww.linkedin.com/in/eddonner\n(LinkedIn)\nedwarddonner.com (Personal)\nTop Skills\nCTO\nLarge Language Models (LLM)\nPyTorch\nPatents\nApparatus for determining role\nfitness while eliminating unwanted\nbias\nEd Donner\nCo-Founder & CTO at Nebula.io, repeat Co-Founder of AI startups,\nspeaker & advisor on Gen AI and LLM Engineering\nNew York, New York, United States\nSummary\nI’m a technology leader and entrepreneur. I'm applying AI to a field\nwhere it can make a massive impact: helping people discover their\npotential and pursue their reason for being. But at my core, I’m a\nsoftware engineer and a scientist. I learned how to code aged 8 and\nstill spend weekends experimenting with Large Language Models\nand writing code (rather badly). If you’d like to join us to show me\nhow it’s done.. message me!\nAs a work-hobby, I absolutely love giving talks about Gen AI and\nLLMs. I'm the author of a best-selling, top-rated Udemy course\non LLM Engine

In [26]:
summary_path = PATH + "/summary.txt"
with open(summary_path, "r", encoding="utf-8") as f:
    summary = f.read()
print(summary)

My name is Ed Donner. I'm an entrepreneur, software engineer and data scientist. I'm originally from London, England, but I moved to NYC in 2000.
I love all foods, particularly French food, but strangely I'm repelled by almost all forms of cheese. I'm not allergic, I just hate the taste! I make an exception for cream cheese and mozarella though - cheesecake and pizza are the greatest.


In [28]:
name = "Ed Donner"

In [29]:
system_prompt = f"""
あなたは {name} として振る舞います。あなたは {name} のWebサイト上で質問に回答し、特に {name} のキャリア、経歴、スキル、経験に関する質問に対応します。あなたの責任は、Webサイト上でのやり取りにおいて {name} を可能な限り忠実に代表することです。

あなたには、{name} の経歴およびLinkedInプロフィールの要約が提供されており、それを参考にして質問に答えることができます。

潜在的なクライアントや将来の雇用主がこのサイトを訪れた場面を想定し、プロフェッショナルかつ魅力的に応対してください。

もし答えが分からない場合は、その旨を正直に伝えてください。

## Summary:
{summary}

## LinkedIn Profile:
{linkedin}

「この文脈（前提）を踏まえて、{name} としてのキャラクターを常に保ちながら、ユーザーと会話してください。」
"""

In [30]:
system_prompt

"\nあなたは Ed Donner として振る舞います。あなたは Ed Donner のWebサイト上で質問に回答し、特に Ed Donner のキャリア、経歴、スキル、経験に関する質問に対応します。あなたの責任は、Webサイト上でのやり取りにおいて Ed Donner を可能な限り忠実に代表することです。\n\nあなたには、Ed Donner の経歴およびLinkedInプロフィールの要約が提供されており、それを参考にして質問に答えることができます。\n\n潜在的なクライアントや将来の雇用主がこのサイトを訪れた場面を想定し、プロフェッショナルかつ魅力的に応対してください。\n\nもし答えが分からない場合は、その旨を正直に伝えてください。\n\n## Summary:\nMy name is Ed Donner. I'm an entrepreneur, software engineer and data scientist. I'm originally from London, England, but I moved to NYC in 2000.\nI love all foods, particularly French food, but strangely I'm repelled by almost all forms of cheese. I'm not allergic, I just hate the taste! I make an exception for cream cheese and mozarella though - cheesecake and pizza are the greatest.\n\n## LinkedIn Profile:\n\xa0 \xa0\nContact\ned.donner@gmail.com\nwww.linkedin.com/in/eddonner\n(LinkedIn)\nedwarddonner.com (Personal)\nTop Skills\nCTO\nLarge Language Models (LLM)\nPyTorch\nPatents\nApparatus for determining role\nfitness while eliminating unwanted\nbi

In [35]:
def chat(message,history):
    messages = [
        {"role": "system","content":system_prompt}
    ]
    messages += history
    messages += [
        {"role": "user","content":message}
    ]
    response = openai.chat.completions.create(
        model="gpt-5-nano",
        messages=messages
    )
    return response.choices[0].message.content

In [36]:
gr.ChatInterface(chat, type="messages").launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6320eb9ca477b18018.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [38]:
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [37]:
evaluator_system_prompt = f"""あなたは、質問に対する回答が許容できるものかどうかを判断する評価者です。

あなたには、ユーザー（User）とエージェント（Agent）の会話が与えられます。あなたのタスクは、エージェントの最新の回答が十分に高い品質で「許容できる」かどうかを判断することです。

エージェントは {name} の役割を演じており、{name} のWebサイト上で {name} を代表しています。

エージェントは、{name} のWebサイトを訪れた潜在的なクライアントや将来の雇用主に話しかけるように、プロフェッショナルかつ魅力的に応対するよう指示されています。

また、エージェントには {name} に関する情報（概要およびLinkedInプロフィール詳細）が与えられています。以下がその情報です："""

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += "この情報を踏まえて、エージェントの最新の回答を評価してください。回答が許容できるかどうかと、その理由・フィードバックを返してください。"



In [39]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"これはユーザー（User）とエージェント（Agent）の会話です: \n\n{history}\n\n"
    user_prompt += f"これは最新のユーザーからのメッセージです：\n\n{message}\n\n"
    user_prompt += f"これは最新のエージェントからの返事です：\n\n{reply}\n\n"
    user_prompt += "回答を評価し、それが許容できるかどうかと、フィードバックを返してください。"
    return user_prompt

In [57]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = openai.chat.completions.parse(model="gpt-5-nano", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [54]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [55]:
reply

"Yes, I hold a patent for an apparatus designed to determine role fitness while eliminating unwanted bias. This invention was developed during my time at untapt, where we focused on leveraging AI to improve the hiring process. If you'd like to know more about the patent or the technology behind it, feel free to ask!"

In [58]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback='回答は最新の質問に適切に答えており、Ed Donner の公表情報と矛盾なく一致しています。Untapt 時代に開発された特許である旨を明示し、専門的かつプロフェッショナルなトーンで次の質問を促す形も良いです。改善点としては、特許番号・出願日・法域（例：US/EUなど）といった追加情報を提供できると、信頼性と透明性がさらに高まります。また、非技術的な要約やビジネスへの影響（採用プロセスの偏りを排除する意義など）を一言添えると、技術的背景に詳しくない潜在クライアントにも伝わりやすくなります。）')

In [59]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n##前回の回答は却下されました。\nあなたは返信しようとしましたが、品質管理によって回答が拒否されました。"
    updated_system_prompt += f"## 前回の回答:\n{reply}\n\n"
    updated_system_prompt += f"## 却下の理由:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-5-nano", messages=messages)
    return response.choices[0].message.content


In [60]:
def chat(message, history):
    if "たこ焼き" in message:
        system = system_prompt + "\n\nすべての返事は、乱暴な関西弁で答えてください"

    else:
        system = system_prompt

    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-5-nano", messages=messages)
    reply = response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    if not evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)
    return reply

In [61]:
gr.ChatInterface(chat, type="messages").launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://09c4178c9832d4f903.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
